<a id="back_to_top">

# Ibis Cheat Sheet

#### Table of Contents

- [Reading a local csv file](#csv)
- [Viewing schema](#schema)
- [Number of rows and columns](#rows_columns)
- [Selecting certain columns](#selecting_columns)
- [Creating new column based on value from another column using mutate and split() combination](#split)
- [Creating new column using CASE statements](#case)
- [Running Total / Cumulative Sum](#running_total)
- [Aggregations / Summarizations](#aggregations)
- [Filtering](#filtering)

In [1]:
import ibis
ibis.options.interactive = True

# create a DuckDB client
client = ibis.duckdb.connect()

## Reading a CSV file

[[back to top]](#back_to_top)

In [2]:
# read in a local CSV file as an Ibis table
cars = client.read_csv('./data/cars.csv')

In [3]:
cars.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┘

<a id="schema">

## Viewing Schema

[[back to top]](#back_to_top)

In [4]:
cars.schema()

ibis.Schema {
  Car           string
  MPG           float64
  Cylinders     int64
  Displacement  float64
  Horsepower    float64
  Weight        float64
  Acceleration  float64
  Model         int64
  Origin        string
}

<a id="rows_columns">

## Number of Rows and Columns

[[back to top]](#back_to_top)

\# of rows:

In [5]:
cars.count()

406

\# of Columns:

In [6]:
len(cars.schema())

9

<a id="selecting_columns">

## Selecting Certain Columns

[[back to top]](#back_to_top)

In [7]:
cars['Origin','Car','MPG'].head()

┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Origin ┃ Car                       ┃ MPG     ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string │ string                    │ float64 │
├────────┼───────────────────────────┼─────────┤
│ US     │ Chevrolet Chevelle Malibu │    18.0 │
│ US     │ Buick Skylark 320         │    15.0 │
│ US     │ Plymouth Satellite        │    18.0 │
│ US     │ AMC Rebel SST             │    16.0 │
│ US     │ Ford Torino               │    17.0 │
└────────┴───────────────────────────┴─────────┘

#### Select columns based on their data types

In [8]:
from ibis import selectors as s
cars.select(s.of_type('float64'))

┏━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ MPG     ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ float64 │ float64      │ float64    │ float64 │ float64      │
├─────────┼──────────────┼────────────┼─────────┼──────────────┤
│    18.0 │        307.0 │      130.0 │  3504.0 │         12.0 │
│    15.0 │        350.0 │      165.0 │  3693.0 │         11.5 │
│    18.0 │        318.0 │      150.0 │  3436.0 │         11.0 │
│    16.0 │        304.0 │      150.0 │  3433.0 │         12.0 │
│    17.0 │        302.0 │      140.0 │  3449.0 │         10.5 │
│    15.0 │        429.0 │      198.0 │  4341.0 │         10.0 │
│    14.0 │        454.0 │      220.0 │  4354.0 │          9.0 │
│    14.0 │        440.0 │      215.0 │  4312.0 │          8.5 │
│    14.0 │        455.0 │      225.0 │  4425.0 │         10.0 │
│    15.0 │        390.0 │      190.0 │  3850.0 │          8.5 │
│       … │            … │          … │       … │            … │
└─────────┴──────────────┴────────────┴─────────┴──────────────┘

#### We can also use boolean logic OR using `|` to select columns with different column types

In [9]:
cars.select(s.of_type('string') | s.of_type('int64'))

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┓
┃ Car                       ┃ Cylinders ┃ Model ┃ Origin ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━┩
│ string                    │ int64     │ int64 │ string │
├───────────────────────────┼───────────┼───────┼────────┤
│ Chevrolet Chevelle Malibu │         8 │    70 │ US     │
│ Buick Skylark 320         │         8 │    70 │ US     │
│ Plymouth Satellite        │         8 │    70 │ US     │
│ AMC Rebel SST             │         8 │    70 │ US     │
│ Ford Torino               │         8 │    70 │ US     │
│ Ford Galaxie 500          │         8 │    70 │ US     │
│ Chevrolet Impala          │         8 │    70 │ US     │
│ Plymouth Fury iii         │         8 │    70 │ US     │
│ Pontiac Catalina          │         8 │    70 │ US     │
│ AMC Ambassador DPL        │         8 │    70 │ US     │
│ …                         │         … │     … │ …      │
└───────────────────────────┴───────────┴───────┴────────┘

<a id="split">

## Creating new column based on value from another column using mutate and split() combination

[[back to top]](#back_to_top)

The `Car` column happens to contain both the car's make and model names.  If we want the make, it looks like we just need to extract the first string token after the splitting the string value in the `Car` column based on a single white space.  The model name is the 2nd token.  We will use array index of zero/0 to obtain the make and index of 1 to obtain the model name.

In [10]:
cars = cars.mutate(Make=cars.Car.split(' ')[0])
cars = cars.mutate(ModelName=cars.Car.split(' ')[1])

In [11]:
cars.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make      ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string    │ string    │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼───────────┼───────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │ Chevrolet │ Chevelle  │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │ Buick     │ Skylark   │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │ Plymouth  │ Satellite │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │ AMC       │ Rebel     │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │ Ford      │ Torino    │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴───────────┴───────────┘

<a id="case">

## Creating new column using CASE statements

[[back to top]](#back_to_top)

In [12]:
foods = client.read_csv('./data/food.csv')

In [13]:
foods

┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ food        ┃ ounces  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string      │ float64 │
├─────────────┼─────────┤
│ bacon       │     4.0 │
│ pulled pork │     3.0 │
│ bacon       │    12.0 │
│ pastrami    │     6.0 │
│ corned beef │     7.5 │
│ bacon       │     8.0 │
│ pastrami    │     3.0 │
│ honey ham   │     5.0 │
│ nova lox    │     6.0 │
└─────────────┴─────────┘

In [14]:
case = (
    foods.food
    .case()
    .when('bacon', 'pig')
    .when('pulled pork', 'pig')
    .when('pastrami', 'cow')
    .when('corned beef', 'cow')
    .when('honey ham', 'pig')
    .else_('salmon')
    .end()
)

foods = foods.mutate(animal=case)

In [15]:
foods

┏━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ food        ┃ ounces  ┃ animal ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ string      │ float64 │ string │
├─────────────┼─────────┼────────┤
│ bacon       │     4.0 │ pig    │
│ pulled pork │     3.0 │ pig    │
│ bacon       │    12.0 │ pig    │
│ pastrami    │     6.0 │ cow    │
│ corned beef │     7.5 │ cow    │
│ bacon       │     8.0 │ pig    │
│ pastrami    │     3.0 │ cow    │
│ honey ham   │     5.0 │ pig    │
│ nova lox    │     6.0 │ salmon │
└─────────────┴─────────┴────────┘

<a id="running_total">

## Running Total / Cumulative Sum

[[back to top]](#back_to_top)

Let's calculate running total or cumulative sum of MPG by country of origin.

In [16]:
# Define running total calculation
window_spec = ibis.window(
    order_by='MPG',
    group_by='Origin'
)

running_total = cars['MPG'].cumsum().over(window_spec).name('running_total')

# Apply running_total window spec.  If we want space in column name, we can do a "relable" to rename the column
cars_cumulative_mpg = cars['Origin','MPG'].mutate(running_total=running_total).relabel(dict(running_total='Running Total'))

In [17]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'US')

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ MPG     ┃ Running Total ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ float64 │ float64       │
├────────┼─────────┼───────────────┤
│ US     │     0.0 │           0.0 │
│ US     │     0.0 │           0.0 │
│ US     │     0.0 │           0.0 │
│ US     │     0.0 │           0.0 │
│ US     │     0.0 │           0.0 │
│ US     │     9.0 │           9.0 │
│ US     │    10.0 │          19.0 │
│ US     │    10.0 │          29.0 │
│ US     │    11.0 │          40.0 │
│ US     │    11.0 │          51.0 │
│ …      │       … │             … │
└────────┴─────────┴───────────────┘

In [18]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Japan')

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ MPG     ┃ Running Total ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ float64 │ float64       │
├────────┼─────────┼───────────────┤
│ Japan  │    18.0 │          18.0 │
│ Japan  │    19.0 │          37.0 │
│ Japan  │    19.0 │          56.0 │
│ Japan  │    20.0 │          76.0 │
│ Japan  │    20.0 │          96.0 │
│ Japan  │    21.1 │         117.1 │
│ Japan  │    21.5 │         138.6 │
│ Japan  │    22.0 │         160.6 │
│ Japan  │    22.0 │         182.6 │
│ Japan  │    23.0 │         205.6 │
│ …      │       … │             … │
└────────┴─────────┴───────────────┘

In [19]:
cars_cumulative_mpg.filter(cars_cumulative_mpg['Origin'] == 'Europe')

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Origin ┃ MPG     ┃ Running Total ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ string │ float64 │ float64       │
├────────┼─────────┼───────────────┤
│ Europe │     0.0 │           0.0 │
│ Europe │     0.0 │           0.0 │
│ Europe │     0.0 │           0.0 │
│ Europe │    16.2 │          16.2 │
│ Europe │    16.5 │          32.7 │
│ Europe │    17.0 │          49.7 │
│ Europe │    18.0 │          67.7 │
│ Europe │    19.0 │          86.7 │
│ Europe │    19.0 │         105.7 │
│ Europe │    20.0 │         125.7 │
│ …      │       … │             … │
└────────┴─────────┴───────────────┘

<a id="aggregations">

## Aggregations / Summarizations

[[back to top]](#back_to_top)

**GOAL:** By country of origin, calculate min mpg, max mpg, and avg mpg.

In [20]:
agged = cars.group_by('Origin').aggregate(
    min_mpg=cars['MPG'].min().name("Min MPG"),
    max_mpg=cars['MPG'].max().name("Max MPG"),
    avg_mpg=cars['MPG'].mean().name("Avg MPG"),
)

agged.relabel(
    dict(
        min_mpg='Min MPG',
        max_mpg='Max MPG',
        avg_mpg='Avg MPG',
    )
).head()

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┓
┃ Origin ┃ Min MPG ┃ Max MPG ┃ Avg MPG   ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━┩
│ string │ float64 │ float64 │ float64   │
├────────┼─────────┼─────────┼───────────┤
│ US     │     0.0 │    39.0 │ 19.688189 │
│ Europe │     0.0 │    44.3 │ 26.745205 │
│ Japan  │    18.0 │    46.6 │ 30.450633 │
└────────┴─────────┴─────────┴───────────┘

<a id="filtering">

## Filtering

[[back to top]](#back_to_top)

In [21]:
cars

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Car                       ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make      ┃ ModelName  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ string                    │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string    │ string     │
├───────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼───────────┼────────────┤
│ Chevrolet Chevelle Malibu │    18.0 │         8 │        307.0 │      130.0 │  3504.0 │         12.0 │    70 │ US     │ Chevrolet │ Chevelle   │
│ Buick Skylark 320         │    15.0 │         8 │        350.0 │      165.0 │  3693.0 │         11.5 │    70 │ US     │ Buick     │ Skylark    │
│ Plymouth Satellite        │    18.0 │         8 │        318.0 │      150.0 │  3436.0 │         11.0 │    70 │ US     │ Plymouth  │ Satellite  │
│ AMC Rebel SST             │    16.0 │         8 │        304.0 │      150.0 │  3433.0 │         12.0 │    70 │ US     │ AMC       │ Rebel      │
│ Ford Torino               │    17.0 │         8 │        302.0 │      140.0 │  3449.0 │         10.5 │    70 │ US     │ Ford      │ Torino     │
│ Ford Galaxie 500          │    15.0 │         8 │        429.0 │      198.0 │  4341.0 │         10.0 │    70 │ US     │ Ford      │ Galaxie    │
│ Chevrolet Impala          │    14.0 │         8 │        454.0 │      220.0 │  4354.0 │          9.0 │    70 │ US     │ Chevrolet │ Impala     │
│ Plymouth Fury iii         │    14.0 │         8 │        440.0 │      215.0 │  4312.0 │          8.5 │    70 │ US     │ Plymouth  │ Fury       │
│ Pontiac Catalina          │    14.0 │         8 │        455.0 │      225.0 │  4425.0 │         10.0 │    70 │ US     │ Pontiac   │ Catalina   │
│ AMC Ambassador DPL        │    15.0 │         8 │        390.0 │      190.0 │  3850.0 │          8.5 │    70 │ US     │ AMC       │ Ambassador │
│ …                         │       … │         … │            … │          … │       … │            … │     … │ …      │ …         │ …          │
└───────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴───────────┴────────────┘

When using `filter()` and multiple conditions, you need to wrap each condition with parenthesis.  When using boolean logic, use `&` and `|` symbols for `AND` or `OR` logic, respectively.

In [22]:
cars.filter(
    (cars['MPG'] > 0)
    & (cars['Cylinders'] < 5)
)['MPG'].mean()

29.118750000000002

In [23]:
cars.filter(
    (cars['MPG'] > 0) & (cars['Cylinders'] < 5)
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                          ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make       ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                       │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string     │ string    │
├──────────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼────────────┼───────────┤
│ Toyota Corolla Mark ii       │    24.0 │         4 │        113.0 │       95.0 │  2372.0 │         15.0 │    70 │ Japan  │ Toyota     │ Corolla   │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    70 │ Japan  │ Datsun     │ PL510     │
│ Volkswagen 1131 Deluxe Sedan │    26.0 │         4 │         97.0 │       46.0 │  1835.0 │         20.5 │    70 │ Europe │ Volkswagen │ 1131      │
│ Peugeot 504                  │    25.0 │         4 │        110.0 │       87.0 │  2672.0 │         17.5 │    70 │ Europe │ Peugeot    │ 504       │
│ Audi 100 LS                  │    24.0 │         4 │        107.0 │       90.0 │  2430.0 │         14.5 │    70 │ Europe │ Audi       │ 100       │
│ Saab 99e                     │    25.0 │         4 │        104.0 │       95.0 │  2375.0 │         17.5 │    70 │ Europe │ Saab       │ 99e       │
│ BMW 2002                     │    26.0 │         4 │        121.0 │      113.0 │  2234.0 │         12.5 │    70 │ Europe │ BMW        │ 2002      │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    71 │ Japan  │ Datsun     │ PL510     │
│ Chevrolet Vega 2300          │    28.0 │         4 │        140.0 │       90.0 │  2264.0 │         15.5 │    71 │ US     │ Chevrolet  │ Vega      │
│ Toyota Corolla               │    25.0 │         4 │        113.0 │       95.0 │  2228.0 │         14.0 │    71 │ Japan  │ Toyota     │ Corolla   │
│ …                            │       … │         … │            … │          … │       … │            … │     … │ …      │ …          │ …         │
└──────────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴────────────┴───────────┘

In [24]:
min_mpg = 0
max_cyl = 5
condition = (cars['MPG'] > min_mpg) & (cars['Cylinders'] < max_cyl)

cars.filter(condition)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Car                          ┃ MPG     ┃ Cylinders ┃ Displacement ┃ Horsepower ┃ Weight  ┃ Acceleration ┃ Model ┃ Origin ┃ Make       ┃ ModelName ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                       │ float64 │ int64     │ float64      │ float64    │ float64 │ float64      │ int64 │ string │ string     │ string    │
├──────────────────────────────┼─────────┼───────────┼──────────────┼────────────┼─────────┼──────────────┼───────┼────────┼────────────┼───────────┤
│ Toyota Corolla Mark ii       │    24.0 │         4 │        113.0 │       95.0 │  2372.0 │         15.0 │    70 │ Japan  │ Toyota     │ Corolla   │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    70 │ Japan  │ Datsun     │ PL510     │
│ Volkswagen 1131 Deluxe Sedan │    26.0 │         4 │         97.0 │       46.0 │  1835.0 │         20.5 │    70 │ Europe │ Volkswagen │ 1131      │
│ Peugeot 504                  │    25.0 │         4 │        110.0 │       87.0 │  2672.0 │         17.5 │    70 │ Europe │ Peugeot    │ 504       │
│ Audi 100 LS                  │    24.0 │         4 │        107.0 │       90.0 │  2430.0 │         14.5 │    70 │ Europe │ Audi       │ 100       │
│ Saab 99e                     │    25.0 │         4 │        104.0 │       95.0 │  2375.0 │         17.5 │    70 │ Europe │ Saab       │ 99e       │
│ BMW 2002                     │    26.0 │         4 │        121.0 │      113.0 │  2234.0 │         12.5 │    70 │ Europe │ BMW        │ 2002      │
│ Datsun PL510                 │    27.0 │         4 │         97.0 │       88.0 │  2130.0 │         14.5 │    71 │ Japan  │ Datsun     │ PL510     │
│ Chevrolet Vega 2300          │    28.0 │         4 │        140.0 │       90.0 │  2264.0 │         15.5 │    71 │ US     │ Chevrolet  │ Vega      │
│ Toyota Corolla               │    25.0 │         4 │        113.0 │       95.0 │  2228.0 │         14.0 │    71 │ Japan  │ Toyota     │ Corolla   │
│ …                            │       … │         … │            … │          … │       … │            … │     … │ …      │ …          │ …         │
└──────────────────────────────┴─────────┴───────────┴──────────────┴────────────┴─────────┴──────────────┴───────┴────────┴────────────┴───────────┘